<a href="https://colab.research.google.com/github/PeteJoh/fpl-ai/blob/master/Minutes_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# MAKE SURE TO CHANGE GW TO NEXT GAME TO BE PLAYED
# SAVE TO GITHUB WHEN FINISHED

In [1]:
#------------------------------------------------------
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
forward_statistics =['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
        'mean assists 3','mean bonus 3', 'mean bps 3','mean creativity 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3', 'mean penalties_missed 3',  'mean threat 3',
       'mean total_points 3','mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

leak_columns = [
    "name",
    "team",
]  # columns that shouldnt be used in training for fear of data leakage


dropped_columns = [
    "season",
    "opponent",
    "match_result",
   # "position",
    "assists",
    "penalties_missed",
    "bonus",
    "bps",
    "clean_sheets",
    "creativity",
    "goals_conceded",
    "goals_scored",
    "ict_index",
    "influence",
    "own_goals",
    "penalties_saved",
    "red_cards",
    "saves",
    "selected",
    "threat",
    "transfers_balance",
    "transfers_in",
    "transfers_out",
    "yellow_cards",
    "team Goal scored",
    "team Goal conceded"
]  # "value",

midfielder_statistics =['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
        'mean assists 3','mean bonus 3', 'mean bps 3','mean creativity 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3', 'mean penalties_missed 3',  'mean threat 3',
       'mean total_points 3','mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

goalkeeper_statistics = ['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'total_points_ex', 'minutes_ex', 'goals_conceded_ex',
       'bonus_ex', 'bps_ex', 'ict_index_ex', 'clean_sheets_ex',
       'red_cards_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
       'mean bonus 3', 'mean bps 3', 'mean clean_sheets 3', 'mean goals_conceded 3',
       'mean ict_index 3',  'mean minutes 3',
       'mean own_goals 3',  'mean penalties_saved 3',
        'mean saves 3',  'mean threat 3',
       'mean total_points 3',
       'mean value 3', 'mean match_result 3', 'std bps 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

statistics =['value', 'position','was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'clean_sheets_ex',
       'yellow_cards_ex','now_cost_ex', 'GW', 'opponent_last_season_position', 'mean assists 3',
       'mean bonus 3', 'mean bps 3', 'mean clean_sheets 3',
       'mean creativity 3', 'mean goals_conceded 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3',
       'mean own_goals 3',
       'mean red_cards 3',  'mean threat 3','mean total_points 3',
       'mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3','mean saves 3','mean assists all',
       'mean bonus all', 'mean bps all', 'mean clean_sheets all',
       'mean creativity all', 'mean goals_conceded all', 'mean goals_scored all',
       'mean ict_index all', 'mean influence all', 'mean minutes all',
       'mean own_goals all',
       'mean red_cards all',  'mean threat all','mean total_points all',
       'mean value all', 'mean match_result all',
       'mean team Goal scored 3','mean team Goal scored all','mean team Goal conceded 3','mean team Goal conceded all',"ratio_goal_scored all","ratio_goal_scored 3",
       'opp mean team Goal scored 3','opp mean team Goal conceded 3','opp mean team Goal scored all','opp mean team Goal conceded all',"opp mean match_result all"]


date_cols=["day_of week","month","hour","week"]

In [3]:
def convert_minutes(val):
    """CONVERTS MINUTES TO A CATEGORICAL OUTPUT"""
    if val > 10:
        return 1
    else:
        return 0

In [4]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.7 MB/s eta 0:00:00


In [5]:
from hyperopt import tpe,hp,fmin,STATUS_OK,Trials
from hyperopt.pyll.base import scope

In [6]:
import pandas as pd
import numpy as np
import warnings
import os


warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import (
    RandomForestClassifier,
    RandomForestRegressor,
    GradientBoostingRegressor,
)
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    confusion_matrix,
    accuracy_score,
    f1_score,
)
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,KFold



In [7]:
# ************** CHANGE GW TO NEXT GW TO BE PLAYED ***************************
gameweek=12
# ****************************************************************************

In [8]:
# RELOAD THIS DATA FOR EACH MODEL

train = pd.read_csv("/content/gdrive/MyDrive/fplpredict/cleaned_dataset/cleaned_previous_seasons.csv",index_col=0)#/content/drive/MyDrive/Fplpredict /cleaned_previous_seasons.csv", index_col=0)
old_gameweek_cleaned = []
for i in range(1, gameweek):
    old_gameweek_cleaned.append(pd.read_csv(f"/content/gdrive/MyDrive/fplpredict/cleaned_dataset/2024-25/GW{i}.csv"))
old_gameweeks = pd.concat(old_gameweek_cleaned)[train.columns]
train = pd.concat([train, old_gameweeks])

# data for current gameweek we want to predict on
test = pd.read_csv(f"/content/gdrive/MyDrive/fplpredict/cleaned_dataset/2024-25/GW{gameweek}.csv", index_col=0)

In [9]:
#CREATE GAMEWEEK PREDICTIONS FOLDER
path = f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}"

# create new single directory
# check whether directory already exists
if not os.path.exists(path):
    os.mkdir(path)
    print("Folder %s created!" % path)
else:
    print("Folder %s already exists" % path)

# data from current season but previous gameweeks

Folder /content/gdrive/MyDrive/fplpredict/predicted_dataset/GW12 created!


In [10]:
train["position"].value_counts()

,count
position,
MID,53123
DEF,42615
FWD,15866
GK,13166
GKP,899


In [11]:
train["position"]=train["position"].replace({"GKP":"GK" })

In [12]:
# MINUTES PREDICTOR STARTS HERE************************************************
# Ensure both DataFrames have the same columns before alignment
# Create the date-related columns in 'test' first
import pandas as pd
from sklearn.model_selection import train_test_split

test["index"] = test["name"] + test["kickoff_time"].astype("str")
test = test.set_index("index")
test["date"] = pd.to_datetime(test["kickoff_time"], format='ISO8601', errors='coerce')
test["day_of week"] = test["date"].dt.day_name()  # Added parentheses to call the function
test["month"] = test["date"].dt.month
test["hour"] = test["date"].dt.hour
test["week"] = test["date"].dt.isocalendar().week
test.drop(["kickoff_time", "date"], axis=1, inplace=True)

# Now process 'train' DataFrame
# Create 'index' column *before* dropping 'kickoff_time'
train["index"] = train["name"] + train["kickoff_time"].astype("str")
train.drop_duplicates("index", keep="last", inplace=True)
train = train.set_index("index")

# Now you can create date features and then drop 'kickoff_time'
train["date"] = pd.to_datetime(train["kickoff_time"], format='ISO8601', errors='coerce')
train["day_of week"] = train["date"].dt.day_name()  # Added parentheses to call the function
train["month"] = train["date"].dt.month
train["hour"] = train["date"].dt.hour
train["week"] = train["date"].dt.isocalendar().week
train.drop(["kickoff_time", "date"], axis=1, inplace=True) # Now safe to drop


#Keep only common columns in both dataframes
common_cols = list(set(train.columns) & set(test.columns))

train = train[common_cols]
test = test[common_cols]



train_copy = train.copy()
test_copy = test.copy()


train["minutes"] = train["minutes"].apply(convert_minutes)

target = train[["minutes", "GW", "position"]]
train.drop(["total_points", "minutes"], axis=1, inplace=True)
test.drop(["total_points", "minutes"], axis=1, inplace=True)
train.drop(dropped_columns, axis=1, inplace=True)
test.drop(dropped_columns, axis=1, inplace=True)


# Convert categorical columns to numerical
for col in train.columns:
    if train[col].dtype == "object":
        if col not in ["team", "name", "position"]:
            train[col] = pd.factorize(train[col])[0]
            test[col] = pd.factorize(test[col])[0]

train["was_home"] = train["was_home"].replace({True: 0, False: 1})
test["was_home"] = test["was_home"].replace({True: 0, False: 1})

train = train[statistics + leak_columns + date_cols]
test = test[statistics + leak_columns + date_cols]

x, val, y, y_val = train_test_split(
    train.drop(leak_columns, axis=1),
    target["minutes"],
    test_size=0.1,
    random_state=0,
)

In [13]:
params={'colsample_bylevel': 0.8070621518153563, 'learning_rate': 0.04765984972709895, 'max_depth': 7, 'reg_lambda': 5, 'scale_pos_weight': 2.5,'subsample': 0.6794390204583894}
model=CatBoostClassifier(**params,cat_features=["position"],random_state=0,early_stopping_rounds=500,use_best_model=True,verbose=500,n_estimators=10000)

In [14]:
model.fit(x, y,eval_set=[(val,y_val)])

0:	learn: 0.6540810	test: 0.6541696	best: 0.6541696 (0)	total: 421ms	remaining: 1h 10m 5s
500:	learn: 0.2968429	test: 0.3208058	best: 0.3207861 (495)	total: 1m 23s	remaining: 26m 30s
1000:	learn: 0.2731598	test: 0.3205606	best: 0.3204785 (905)	total: 2m 48s	remaining: 25m 13s
1500:	learn: 0.2535459	test: 0.3211767	best: 0.3202903 (1134)	total: 4m 11s	remaining: 23m 42s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.3202903271
bestIteration = 1134

Shrink model to first 1135 iterations.


In [15]:
print(confusion_matrix(model.predict(val), y_val))
print(
    f"starting Accuracy score {accuracy_score(model.predict(val), y_val)}"
)

print(
    f" starting f1 score: {f1_score(model.predict(val), y_val)}"
)

feature_importance = pd.DataFrame(
    {"column": x.columns, "imp": model.feature_importances_}
).sort_values(
    "imp", ascending=False
)

[[5525  376]
 [1410 4004]]
starting Accuracy score 0.8421564295183385
 starting f1 score: 0.8176434551766387


In [ ]:
feature_importance.head(50)

In [ ]:
feature_importance.tail(20)

In [18]:
y_val

,minutes
index,
James Tarkowski2022-11-05 17:30:00+00:00,1
Matej Vydra2021-01-27 18:00:00+00:00,1
Scott McTominay2022-02-12 12:30:00+00:00,1
Noha Lemina2024-02-25 13:30:00+00:00,0
Jed Steer2021-04-25 18:00:00+00:00,0
...,...
Tyler Onyango2022-05-11 18:45:00+00:00,0
Mbwana Samatta2021-03-03 18:00:00+00:00,0
Mathew Ryan2022-04-02 14:00:00+00:00,0


In [19]:
test_copy["minutes"] = model.predict(test.drop(leak_columns, axis=1))
test_copy[leak_columns + ["minutes"]].to_csv(
    f"minutes.csv"
)

In [20]:
predicted_minutes=model.predict(val)
val_=pd.DataFrame({"ind":val.index,"actul_minutes":y_val,"predicted_minutes":predicted_minutes,"position":val["position"]})

In [21]:
print(f"STARTING GOALKEEPERS PERFORMANCE!!!")
a=val_[val_["position"]=="GK"]["actul_minutes"]
b=val_[val_["position"]=="GK"]["predicted_minutes"]
print(confusion_matrix(a,b))
print(
    f"starting Accuracy score {accuracy_score(a,b)}"
)

print(
    f" starting f1 score: {f1_score(a,b)}"
)

STARTING GOALKEEPERS PERFORMANCE!!!
[[859  39]
 [ 24 324]]
starting Accuracy score 0.949438202247191
 starting f1 score: 0.9113924050632911


In [22]:
print(f"STARTING DEFENDERS PERFORMANCE!!!")
a=val_[val_["position"]=="DEF"]["actul_minutes"]
b=val_[val_["position"]=="DEF"]["predicted_minutes"]
print(confusion_matrix(a,b))
print(
    f"starting Accuracy score {accuracy_score(a,b)}"
)

print(
    f" starting f1 score: {f1_score(a,b)}"
)

STARTING DEFENDERS PERFORMANCE!!!
[[1805  527]
 [ 146 1323]]
starting Accuracy score 0.822941331228624
 starting f1 score: 0.797228080747213


In [23]:
print(f"STARTING MIDFIELDERS PERFORMANCE!!!")
a=val_[val_["position"]=="MID"]["actul_minutes"]
b=val_[val_["position"]=="MID"]["predicted_minutes"]
print(confusion_matrix(a,b))
print(
    f"starting Accuracy score {accuracy_score(a,b)}"
)

print(
    f" starting f1 score: {f1_score(a,b)}"
)

STARTING MIDFIELDERS PERFORMANCE!!!
[[2154  648]
 [ 154 1852]]
starting Accuracy score 0.8331946755407654
 starting f1 score: 0.8220150909897914


In [24]:
print(f"STARTING FORWARDS PERFORMANCE!!!")
a=val_[val_["position"]=="FWD"]["actul_minutes"]
b=val_[val_["position"]=="FWD"]["predicted_minutes"]
print(confusion_matrix(a,b))
print(
    f"starting Accuracy score {accuracy_score(a,b)}"
)

print(
    f" starting f1 score: {f1_score(a,b)}"
)

STARTING FORWARDS PERFORMANCE!!!
[[707 196]
 [ 52 505]]
starting Accuracy score 0.8301369863013699
 starting f1 score: 0.8028616852146264


In [25]:
test_copy[test_copy["minutes"]==1]["position"].value_counts()

,count
position,
MID,160
DEF,111
FWD,37
GKP,21


In [26]:
train = train_copy[train_copy["minutes"] > 0]
test = test_copy[test_copy["minutes"] > 0]

# MINUTES PREDICTION ENDS HERE - MAKE SURE TO SAVE MINUTES PREDICTIONS AT THE BOTTOM OF THE SCRIPT

In [27]:
test_copy[test_copy["position"]=="DEF"][["name","team","minutes"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/defendetrs_minutes.csv")
test_copy[test_copy["position"]=="GKP"][["name","team","minutes"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/goalkeepers_minutes.csv")
test_copy[test_copy["position"]=="MID"][["name","team","minutes"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/midfielders_minutes.csv")
test_copy[test_copy["position"]=="FWD"][["name","team","minutes"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/forwards_minutes.csv")